# 2018 OKFL Post-Season Review
This script leverages the MyFantasyLeague API to access the OKFL data for 2018. Data is cleaned, merged, and exported as a series of flat files to be used in Tableau visualizations.

## Init routines

In [1]:
# Import libraries
import numpy as np
import pandas as pd
from MFL.API import GetPlayers, WeeklyResults, PlayerScores, DraftResults, Projections

MFL package version 0.6.0
  API scripts
  Utility scripts


In [2]:
# Create manual lists

## ETL
Download, clean, and parse:
- player data (current)
- player points (all players)
- player projected points (all players)
- weekly matchup results
    - starting lineups
    - optimal lineups
    - matchup outcomes
- draft information

### Download and clean data

In [3]:
# Download player data
players = GetPlayers(season = '2018')
players.head()

,player_id,name,position,status,team
0,0151,"Bills, Buffalo",TMWR,NaN,BUF
1,0152,"Colts, Indianapolis",TMWR,NaN,IND
2,0153,"Dolphins, Miami",TMWR,NaN,MIA
3,0154,"Patriots, New England",TMWR,NaN,NEP
4,0155,"Jets, New York",TMWR,NaN,NYJ


In [4]:
# Download player scores
scores = PlayerScores(league_id = '27378', season = '2018')
scores.head()

,player_id,points,week,season
0,8062,50.3,1,2018
1,4925,37.6,1,2018
2,13116,36.3,1,2018
3,12801,35.3,1,2018
4,13132,34.1,1,2018


In [5]:
# Download weekly matchup results

results = WeeklyResults(league_id = '27378', season = '2018')
results.head()

C:\Users\Richard\Documents\OKFL\OKFL - Tableau\MFL\API.py:233: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  results = pd.concat([results, r])
C:\Users\Richard\Documents\OKFL\OKFL - Tableau\MFL\API.py:241: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  results = pd.concat([results, r])


,season,week,game,owner_id,isHome,starters,nonstarters,optimal,team_score,result,opt_pts,player
0,2018,1,1,0007,0,"7813,13130,8658,9988,10983,12652,13189,6789,0522,","13617,12152,11668,13116,","13116,11668,12652,0522,6789,13189,8658,9988,12...",105.9,W,143.7,"[{'status': 'starter', 'id': '7813', 'shouldSt..."
1,2018,1,1,0001,1,"10700,13138,8670,12181,12186,9902,11244,12956,...","11248,13607,12930,0527,8360,","10700,9902,0527,12186,8670,12956,11248,8360,12...",75.7,L,110.3,"[{'status': 'starter', 'id': '10700', 'shouldS..."
0,2018,1,2,0008,0,"4925,12634,12171,12205,11232,9884,11250,10976,...","13188,9427,9898,7877,","4925,12171,9884,10976,0518,13188,12634,9427,11...",109.9,L,116.6,"[{'status': 'starter', 'id': '4925', 'shouldSt..."
1,2018,1,2,0005,1,"7394,12151,13132,10527,11671,11938,10312,9686,...","9918,12141,11239,13164,","13132,7394,11671,0528,9686,10312,9918,12151,13...",151.7,W,173.8,"[{'status': 'starter', 'id': '7394', 'shouldSt..."
0,2018,1,3,0003,0,"5848,11660,10729,11705,13604,10738,9831,8359,0...","10389,12175,13192,12157,","5848,13604,0511,9831,8359,11705,10389,10729,10...",105.5,L,117.5,"[{'status': 'starter', 'id': '5848', 'shouldSt..."


In [6]:
# Download projections
projections = Projections(league_id = '27378', season = '2018')
projections.head()

,season,week,player_id,proj_points
0,2018,1,5848,28.1
1,2018,1,10703,22.7
2,2018,1,9431,22.4
3,2018,1,7401,22.4
4,2018,1,7836,22.2


In [7]:
# Download draft results
draft = DraftResults(league_id='27378', season='2018')
draft.astype({'drafted_by' : 'str', 
              'pick' : 'int32', 
              'player_id' : 'str', 
              'round' : 'int32'}, 
             copy = True)

draft.head()

,drafted_by,pick,player_id,round
0101,0002,01,12150,01
0102,0009,02,11192,01
0103,0004,03,12625,01
0104,0008,04,12171,01
0105,0003,05,13604,01


### Clean and Blend data

In [8]:
# Parse out the weekly results data
# initialize dataframes and lists of players
row_data = pd.DataFrame(columns = ['season', 'week', 'game', 'owner_id'])
starters = pd.DataFrame(columns = ['season', 'week', 'game', 'owner_id', 'player_id', 'starter'])
nonstarters = pd.DataFrame(columns = ['season', 'week', 'game', 'owner_id', 'player_id', 'starter'])
optimals = pd.DataFrame(columns = ['season', 'week', 'game', 'owner_id', 'player_id', 'optimal'])
outcomes = pd.DataFrame(columns = ['season', 'week', 'game', 'owner_id', 'outcome'])

row_data = pd.DataFrame.from_dict(data = {'season' : results['season'].tolist(),
                                          'week' : results['week'].tolist(),
                                          'game' : results['game'].tolist(),
                                          'owner_id' : results['owner_id'].tolist(),
                                          'outcome' : results['result']})

starter_lists = results['starters'].tolist()
nonstarter_lists = results['nonstarters'].tolist()
optimal_lists = results['optimal'].tolist()

# create dataframes of starters, nonstarters, and optimal starters for each row
rows = range(len(results))
for r in rows:
    #print('row: ' + str(r))
    season = row_data['season'].iloc[r]
    week = row_data['week'].iloc[r]
    game = row_data['game'].iloc[r]
    owner_id = row_data['owner_id'].iloc[r]
    outcome = row_data['outcome'].iloc[r]
    
    # assign starters
    row_starters = starter_lists[r][0:-1]
    slist = row_starters.split(',')
    pdf = pd.DataFrame.from_dict(data = {'season' : season,
                                         'week' : week,
                                         'game' : game,
                                         'owner_id' : owner_id,
                                         'player_id' : slist,
                                         'starter' : True})
    starters = pd.concat([starters, pdf])
    
    # assign nonstarters
    row_nonstarters = nonstarter_lists[r][0:-1]
    slist = row_nonstarters.split(',')
    pdf = pd.DataFrame.from_dict(data = {'season' : season,
                                         'week' : week,
                                         'game' : game,
                                         'owner_id' : owner_id,
                                         'player_id' : slist,
                                         'starter' : False})
    nonstarters = pd.concat([nonstarters, pdf])
    #pdf = pd.DataFrame.from_dict(data = {'season' : season, 'week' : week, 'game' : game, 'player_id' : slist})
    
    # assign optimal starters
    optimal_starters = optimal_lists[r][0:-1]
    slist = optimal_starters.split(',')
    pdf = pd.DataFrame.from_dict(data = {'season' : season,
                                         'week' : week,
                                         'game' : game,
                                         'owner_id' : owner_id,
                                         'player_id' : slist,
                                         'optimal' : True})
    optimals = pd.concat([optimals, pdf])
    
    # create matchup data
    pdf = pd.DataFrame(data = {'season' : season, 
                                         'week' : week, 
                                         'game' : game, 
                                         'owner_id' : owner_id, 
                                         'outcome' : outcome}, index = [r])
    outcomes = pd.concat([outcomes, pdf])
    
# merge starters and nonstarters
starter_data = pd.concat([starters, nonstarters])
starter_data['week'] = starter_data['week'].astype('int64')
optimals['week'] = optimals['week'].astype('int64')

# add optimal flag
starter_data = pd.merge(starter_data, optimals,
                        how = 'left',
                        left_on = ['season', 'week', 'game', 'owner_id', 'player_id'],
                        right_on = ['season', 'week', 'game', 'owner_id', 'player_id'])
starter_data.fillna(value = False, inplace = True)
starter_data.head()
outcomes.head()

,season,week,game,owner_id,outcome
0,2018,1,1,0007,W
1,2018,1,1,0001,L
2,2018,1,2,0008,L
3,2018,1,2,0005,W
4,2018,1,3,0003,L


In [9]:
# Add draft index to player data
draft['draft_id'] = draft.index.to_series()
d = draft[['player_id', 'draft_id']]

detailed_players = pd.merge(players, d, 
                            how = 'left', 
                            left_on = 'player_id', 
                            right_on = 'player_id')

detailed_players['draft_id'].fillna('0000', inplace = True)
detailed_players = detailed_players[detailed_players['position'].isin(['QB', 'RB', 'WR', 'TE', 'PK', 'Def'])]
detailed_players.astype({'player_id' : 'str', 
                         'name' : 'str', 
                         'position' : 'category', 
                         'team' : 'category', 
                         'draft_id' : 'str'}, 
                        copy = True)

detailed_players.head()

,player_id,name,position,status,team,draft_id
192,0501,"Bills, Buffalo",Def,NaN,BUF,0000
193,0502,"Colts, Indianapolis",Def,NaN,IND,0000
194,0503,"Dolphins, Miami",Def,NaN,MIA,0000
195,0504,"Patriots, New England",Def,NaN,NEP,1301
196,0505,"Jets, New York",Def,NaN,NYJ,0000


In [10]:
# Add starter data to scores
detailed_scores = pd.merge(scores, starter_data, 
                           how = 'outer', 
                           left_on = ['player_id', 'season', 'week'], 
                           right_on = ['player_id', 'season', 'week'])

detailed_scores['owner_id'].fillna('0000', inplace = True)
detailed_scores['starter'].fillna(False, inplace = True)
detailed_scores['game'].fillna(0, inplace = True)
detailed_scores.astype({'player_id' : 'str', 
                        'points' : 'float', 
                        'week' : 'int32', 
                        'season' : 'str', 
                        'game' : 'int32', 
                        'owner_id' : 'str'}, 
                       copy = True)
detailed_scores.head()

,player_id,points,week,season,game,owner_id,starter,optimal
0,8062,50.3,1,2018,5.0,0010,False,True
1,4925,37.6,1,2018,2.0,0008,True,True
2,13116,36.3,1,2018,1.0,0007,False,True
3,12801,35.3,1,2018,3.0,0009,True,True
4,13132,34.1,1,2018,2.0,0005,True,True


## Export flat files for Tableau

In [12]:
# Export cleaned data for use in Tableau dashboard
#   2018 Post Season
detailed_scores.to_csv('detailed_scores.csv')
detailed_players.to_csv('detailed_players.csv')
draft.to_csv('draft.csv')
outcomes.to_csv('outcomes.csv')

print('Output files to csv.')

Output files to csv.
